# 构造表层点位和剖面点位信息

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import fiona
from pathlib import Path

In [2]:

def table_to_shp(input_file, output_file, lon_col, lat_col, input_crs=4326, output_crs=4544):
    # 设置fiona支持utf-8编码
    fiona.supported_drivers['ESRI Shapefile'] = 'rw'

    # 读取输入文件,如果是DataFrame，则直接使用  
    if isinstance(input_file, pd.DataFrame):
        df = input_file
    else:
        _, file_extension = os.path.splitext(input_file)
        if file_extension.lower() == '.xlsx':
            df = pd.read_excel(input_file)
        elif file_extension.lower() == '.csv':
            df = pd.read_csv(input_file, encoding='utf-8')
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")

    # 创建几何列
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    
    # 创建GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=f"EPSG:{input_crs}")
    
    # 如果需要,转换坐标系
    if input_crs != output_crs:
        gdf = gdf.to_crs(epsg=output_crs)
    
    # 保存为shapefile,使用utf-8编码
    gdf.to_file(output_file, driver="ESRI Shapefile", encoding='utf-8')

    print(f"Shapefile已保存至: {output_file}")


In [3]:
# 母岩编码转换
my_dict ={'01': '第四纪松散沉积物','02': '花岗岩','03': '流纹岩','04': '闪长岩','05': '安山岩','06': '正长岩','07': '粗面岩','08': '辉长岩','09': '玄武岩','10': '橄榄岩',
              '11': '脉岩','12': '块集岩','13': '火山角砾岩','14': '凝灰岩','15': '角砾岩','16': '砾岩','17': '砂岩','18': '页岩','19': '化学石灰岩','20': '生物石灰岩','21': '白云岩',
              '22': '片麻岩','23': '石英岩','24': '板岩','25': '结晶片岩','26': '大理岩','27': '泥岩','99': '其他'
              }
def my_transformer(x):
    if x in my_dict:
        return my_dict[x]
    else:
        pass
# 母质编码转换
mz_dict = {'AS': '风积沙','LO': '原生黄土','LOP': '黄土状物质(次生黄土)','LI': '残积物','LG': '坡积物','MA': '洪积物','FL': '冲积物','PY': '海岸沉积物','AL': '湖沉积物',
           'VA': '河流沉积物','CO': '火成碎屑沉积物','WE': '冰川沉积物','SA': '有机沉积物','CD': '崩积物','QR': '红黏土','OT': '其他'}
def mz_transformer(x):
    if x in mz_dict:
        return mz_dict[x]
    else:
        pass
# 地形编码转换



In [4]:
base_data = r"F:\collection_spb_info\GJ\QZ\base_info_20241219_1194.xlsx"
img_data = r"F:\collection_spb_info\GJ\QZ\img_info_20241219_48258.xlsx"
pm_data = r"F:\collection_spb_info\GJ\QZ\pm_info_20241219_103.xlsx"
ldtj_data = r"F:\collection_spb_info\GJ\QZ\ldtj_info_20241219_1194.xlsx"
save_path = os.path.dirname(base_data)

In [5]:
base_df = pd.read_excel(base_data)
img_df = pd.read_excel(img_data)
ldtj_df = pd.read_excel(ldtj_data)
pm_df = pd.read_excel(pm_data)

In [6]:
# 获取my mz
my_mz = pd.merge(base_df[['ydbh','ydlb','dwjd','dwwd','tl','yl','ts','tz']], ldtj_df[['ydbh','tdlylx','dxbw','my','myqt','mz']], on=['ydbh'], how='left')

In [7]:
# 获取剖面tz
pm_tz = pd.merge(my_mz,pm_df[['ydbh','fsxtl', 'fsxyl', 'fsxts', 'fsxtz']],on=['ydbh'],how='left')

In [8]:
# 读取数据
points_df = pm_tz
images_df = img_df

# 合并数据
merged_df = pd.merge(points_df, images_df, left_on='ydbh', right_on='glbh', how='outer')

# 为重复的wjfl添加序号
merged_df['wjfl_count'] = merged_df.groupby(['ydbh', 'wjfl']).cumcount() + 1
merged_df['wjfl_numbered'] = merged_df['wjfl'].astype(str) + '_' + merged_df['wjfl_count'].astype(str)

# 创建透视表
pivot_df = merged_df.pivot(index='ydbh', columns='wjfl_numbered', values='wjlj')

# 重命名列，加上"T"前缀
pivot_df.columns = ['T' + str(col) for col in pivot_df.columns]

# 重置索引，使ydbh成为一个普通列
pivot_df.reset_index(inplace=True)

# 将原始点位信息与新的透视表合并
final_df = pd.merge(points_df, pivot_df, on='ydbh', how='left')

# 保存结果
final_df.to_csv(os.path.join(save_path,'img_merged_result.csv'), index=False)

print(final_df.head())
print(final_df.columns)

# 输出一些统计信息，帮助理解数据
print("\n原始点位数量:", len(points_df))
print("原始图片数量:", len(images_df))
print("最终结果行数:", len(final_df))
print("每个ydbh的平均图片数:", merged_df.groupby('ydbh')['wjfl'].count().mean())
print("图片类型(wjfl)的唯一值:", merged_df['wjfl'].unique())
print("最大的wjfl重复次数:", merged_df.groupby(['ydbh', 'wjfl']).size().max())

               ydbh  ydlb        dwjd       dwwd   tl     yl       ts     tz  \
0  5201810101000001     0  106.448557  26.494108  水稻土  潴育水稻土      鳝泥田    黄泥田   
1  5201810101000017     0  106.353622  26.483658  水稻土  脱潜水稻土     黄灰泥田  干鸭屎泥田   
2  5201810101000028     0  106.380499  26.730040  水稻土  潴育水稻土      鳝泥田    黄泥田   
3  5201810101000032     0  106.342862  26.604709  水稻土  潴育水稻土      灰泥田   大眼泥田   
4  5201810101000043     0  106.255536  26.603190  黄棕壤   暗黄棕壤  灰泥质暗黄棕壤    NaN   

   tdlylx dxbw  ... T1626_7 T1626_8 T1626_9 T1631_1 T1631_2 T1631_3 T1631_4  \
0     101   LS  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1     101   LS  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2     101   LS  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3     101   LS  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4     101   MS  ...     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

  T1631_5 T1631_6 T1631_7  
0     NaN     Na

In [9]:
# 翻译对应的my mz
final_df['my'] = final_df['my'].apply(my_transformer)
final_df['mz'] = final_df['mz'].apply(mz_transformer)

In [10]:
final_df.sample(1)

,ydbh,ydlb,dwjd,dwwd,tl,yl,ts,tz,tdlylx,dxbw,...,T1626_7,T1626_8,T1626_9,T1631_1,T1631_2,T1631_3,T1631_4,T1631_5,T1631_6,T1631_7
1183,5201810404000299,0,106.439363,26.557377,石灰（岩）土,红色石灰土,红色石灰壤土,NaN,404,MS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
input_df = final_df
output_file = r"F:\cache_data\shp_file\qz\qz_sp_img_info.shp"
lon_col = 'dwjd'
lat_col = 'dwwd'
input_crs = 4326
output_crs = 4545

table_to_shp(input_df, output_file, lon_col, lat_col, input_crs, output_crs)

Shapefile已保存至: F:\cache_data\shp_file\qz\qz_sp_img_info.shp


In [9]:


# 合并数据
merged_df = pd.merge(base_df, img_df, left_on='ydbh', right_on='glbh', how='right')

# 处理重复项：对于每个ydbh和wjfl的组合，只保留第一个出现的记录
merged_df = merged_df.sort_values('wjfl').drop_duplicates(['ydbh', 'wjfl'], keep='first')

# 创建透视表
pivot_df = merged_df.pivot(index='ydbh', columns='wjfl', values='wjlj')

# 重命名列，加上"T"前缀
pivot_df.columns = ['T' + str(col) for col in pivot_df.columns]

# 重置索引，使ydbh成为一个普通列
pivot_df.reset_index(inplace=True)

# 将原始点位信息与新的透视表合并
final_df = pd.merge(base_df[['ydbh', 'dwjd', 'dwwd']], pivot_df, on='ydbh', how='left')


In [22]:
# 保存结果
final_df.to_excel(r"C:\Users\Runker\Desktop\dy_point.xlsx",index=False)

In [21]:
final_df

,ydbh,dwjd,dwwd,T1100,T1201,T1202,T1203,T1204,T1209,T1300,...,T1611,T1612,T1613,T1621,T1622,T1623,T1624,T1625,T1626,T1631
0,5227010101000001,107.235946,26.126647,NaN,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-20/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5227010101000006,107.353408,26.223480,NaN,https://sanpu.iarrp.cn/ssp-dccy/2023-10-27/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-27/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-27/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-27/522...,NaN,https://sanpu.iarrp.cn/ssp-dccy/2023-10-27/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5227010101000008,107.693136,26.202125,https://sanpu.iarrp.cn/ssp-dccy/2023-10-23/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-23/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-23/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-23/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5227010101000011,107.577319,26.080480,NaN,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-25/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-07/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-25/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5227010101000013,107.420308,26.291907,NaN,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-30/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,5227010404000850,107.379807,26.022086,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-28/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,5227010404000927,107.724423,26.276913,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-21/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,5227010404001083,107.523825,26.232622,NaN,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-11/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-11/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1157,5227010404001085,107.438007,26.253052,NaN,https://sanpu.iarrp.cn/ssp-dccy/2024-03-11/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,https://sanpu.iarrp.cn/ssp-dccy/2024-03-11/522...,https://sanpu.iarrp.cn/ssp-dccy/2023-10-29/522...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
merged_df.to_excel(r"C:\Users\Runker\Desktop\dy_point.xlsx",index=False)

In [ ]:



# 创建新的要素类
arcpy.env.overwriteOutput = True
arcpy.management.CreateFeatureclass(gdb_path, feature_class_name, "POINT", spatial_reference=4326)

# 添加字段
arcpy.management.AddField(os.path.join(gdb_path, feature_class_name), "点位编码", "TEXT")
arcpy.management.AddField(os.path.join(gdb_path, feature_class_name), "图片链接和类型", "TEXT")

# 创建插入光标
insert_cursor = arcpy.da.InsertCursor(os.path.join(gdb_path, feature_class_name), ["SHAPE@XY", "点位编码", "图片链接和类型"])

# 插入数据
for idx, row in merged_df.iterrows():
    point = (row['LON'], row['LAT'])
    insert_cursor.insertRow([point, row['点位编码'], row['图片链接和类型']])

# 清理
del insert_cursor

print("Feature class created successfully with images links and types.")


In [13]:
a = img_df['wjfl'].value_counts().to_dict()

In [18]:
b = [_ for _ in a.keys()]

In [20]:
b.sort()

In [21]:
b

[1100,
 1201,
 1202,
 1203,
 1204,
 1209,
 1300,
 1301,
 1302,
 1303,
 1304,
 1400,
 1401,
 1500,
 1501,
 1502,
 1611,
 1612,
 1613,
 1621,
 1622,
 1623,
 1624,
 1625,
 1626,
 1631]

In [ ]:
img_df.colu

In [2]:
import arcpy
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.svm import SVR,SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,roc_curve,precision_recall_curve
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
import shutil

In [10]:

# 设置输入文件夹和目标数据库的路径
input_folder = r"D:\ArcGISProjects\workspace\duyun\tif_file"  # TIFF文件所在的文件夹
output_gdb = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb"  # 目标数据库
reference_raster = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb\DEM"  # 参考栅格文件

arcpy.env.workspace = input_folder  # 设置工作空间为输入文件夹

# 从参考栅格文件读取投影信息
ref_desc = arcpy.Describe(reference_raster)
new_projection = ref_desc.spatialReference

# 遍历文件夹中的所有TIF文件
for raster in arcpy.ListRasters("*", "TIF"):
    if raster == 'dem.tif':
        pass
    else:
        # 定义输出路径
        out_raster = os.path.join(output_gdb, os.path.splitext(raster)[0])

        # 将栅格数据复制到数据库中
        # arcpy.CopyRaster_management(raster, out_raster)

        # # 转换投影
        # arcpy.ProjectRaster_management(out_raster, out_raster, new_projection)
        print(raster)



AH.tif
ASP1.tif
CD.tif
CI.tif
CNBL.tif
CND.tif
LSF.tif
PLC.tif
PRC.tif
RSP.tif
SLOPE1.tif
TCA.tif
TWI.tif
VD.tif


In [3]:

# 将两列数据一一对应组成字典并去重
data = pd.DataFrame(data)
result = data.groupby('BM')['土类'].apply(lambda x: list(x.unique())).to_dict()

print(result)


NameError: name 'data' is not defined

In [ ]:
# 水田 1  自然土 34  非自然土 25
dl = "DL"
dem = "DEM"
dz = "DZ"
som = "SOM"
ph = "PH"

# 水稻土 101  黄棕壤102  黄壤103 石灰土104 紫色土105

Con("DL"==1,101,0)
Con("DL"==2,Con("DZ"==1,105,Con((("DZ"==6)|("DZ"==7))&("PH">=7),104,Con("DEM">=1450,102,103))))
Con((("DL"==3)|("DL"==4)),Con("DZ"==1,105,Con((("DZ"==6)|("DZ"==7))&("PH">=7),104,Con("DEM">=1450,102,103))))
Con("DL"==5,Con("DEM">=1450,102,103))
Int("con_SDT1") + Int("con_L4") + Int("con_L3") + Int("con_L2")
Con(IsNull("con_SDT1"), 0, "con_SDT1")+Con(IsNull("con_L4"), 0, "con_L4")+Con(IsNull("con_L3"), 0, "con_L3")+Con(IsNull("con_L2"), 0, "con_L2")


Con("DL"==1,101,Con("DL"==2,Con("DZ"==1,105,Con((("DZ"==6)|("DZ"==7))&("PH">=7),104,Con("DEM">=1450,102,103))),Con((("DL"==3)|("DL"==4)),Con("DZ"==1,105,Con((("DZ"==6)|("DZ"==7))&("PH">=7),104,Con("DEM">=1450,102,103))),Con("DL"==5,Con("DEM">=1450,102,103)))))


In [ ]:
# 结果表达式
Con("con_sd_raste23"==101,100+"TL_SDT_pred",Con("con_sd_raste23"==102,200+"TL_HZR_pred",Con("con_sd_raste23"==103,300+"TL_HR_pred",Con("con_sd_raste23"==104,400+"TL_sht_pred",Con("con_sd_raste23"==105,500+"TL_ZST_pred")))))

In [ ]:
# 导入所需库
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import tree
import graphviz 

# 生成虚拟训练数据
X, y = make_classification(n_samples=1000, n_features=10, n_classes=5, random_state=1)

# 划分训练数据与测试数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2) 

# 通过网格搜索找到最优超参数
params = {'criterion':['gini','entropy'], 'max_depth':list(range(2,10))} 
grid_tree = GridSearchCV(DecisionTreeClassifier(), params, cv=5)
grid_tree.fit(X_train, y_train)

# 使用最优模型参数构建决策树  
clf = DecisionTreeClassifier(**grid_tree.best_params_)
clf.fit(X_train, y_train)

# 模型评估
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

# 可视化决策树
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=feature_names)  
graph = graphviz.Source(dot_data)  
graph.render("decision_tree") 

# 生成决策树图片
graph.view()

In [11]:
import os
import shutil


In [25]:
head_number_1 = 38
head_number_2 = 53
tail_number_1 = 81
tail_number_2 = 94
h48_head_str = "G48G0"
# 定义保存路径
save_path = r"C:\Users\Runker\Desktop\test"
copy_path = r"C:\Users\Runker\Desktop\out"
# 判断是否存在这个文件夹
for inde_1 in range(head_number_1,head_number_2+1):
    for inde_2 in range(tail_number_1,tail_number_2+1):
        number = h48_head_str+str(inde_1).zfill(2) +"0"+str(inde_2).zfill(2)
        # 判断是否存在这个文件夹,
        if not os.path.exists(os.path.join(save_path,number)):
            pass
        # 否则，则拷贝这个文件夹及文件夹内的所有内容
        else:
            if os.path.exists(os.path.join(copy_path,number)):
                print("已存在")
            else:
                shutil.copytree(os.path.join(save_path,number),os.path.join(copy_path,number))
        inde_2+=1
    inde_1+=1

# if not os.path.exists(save_path):
#     os.makedirs(save_path)

 

已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在
已存在


In [19]:
# 输入一个数字
num = 2

# 将数字转换为字符串，并使用zfill方法在前面补零
formatted_num = str(num).zfill(2)

# 打印结果
print("处理后的数字:", formatted_num)


处理后的数字: 02


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

# 模拟数据准备
X = np.random.rand(1000, 30)  # 特征矩阵
y = np.random.randint(0, 5, (1000, 3))  # 假设有3个目标，每个目标5个类别

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建多输出分类器
forest = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

# 训练模型
multi_target_forest.fit(X_train, y_train)

# 预测
y_pred = multi_target_forest.predict(X_test)

# 评估模型
accuracy = [accuracy_score(y_test[:, i], y_pred[:, i]) for i in range(y_test.shape[1])]
for i, acc in enumerate(accuracy, 1):
    print(f"目标{i}的预测准确率：{acc}")


目标1的预测准确率：0.2
目标2的预测准确率：0.18
目标3的预测准确率：0.2


In [3]:
X

array([[0.81272565, 0.61270296, 0.51493628, ..., 0.54577103, 0.97899406,
        0.8483165 ],
       [0.66085331, 0.81709889, 0.18757415, ..., 0.69313085, 0.84574926,
        0.71354819],
       [0.67745215, 0.86022125, 0.84607136, ..., 0.55459895, 0.3854412 ,
        0.60852781],
       ...,
       [0.91083249, 0.8603101 , 0.60459647, ..., 0.95246068, 0.02794886,
        0.66453706],
       [0.75553532, 0.06581614, 0.32612173, ..., 0.55834671, 0.98624542,
        0.84796333],
       [0.74762973, 0.38243898, 0.3698287 , ..., 0.72948439, 0.52899044,
        0.45648462]])

In [5]:
y

array([[3, 2, 2],
       [0, 1, 1],
       [4, 1, 2],
       ...,
       [3, 1, 0],
       [1, 2, 1],
       [1, 2, 1]])

In [8]:
X_train[0]

array([0.57634612, 0.37673869, 0.18348528, 0.23279418, 0.54432573,
       0.05866634, 0.97664044, 0.17842245, 0.13839346, 0.55246262,
       0.91528041, 0.11060267, 0.32831095, 0.91190363, 0.7540211 ,
       0.88320343, 0.90743972, 0.83624756, 0.9404942 , 0.31488327,
       0.72391054, 0.96117635, 0.79032238, 0.21058872, 0.92915583,
       0.35396476, 0.29209955, 0.87249801, 0.05167923, 0.94846948])

In [10]:
y_train[0]

array([3, 0, 3])

In [1]:
import pandas as pd
import os

In [2]:
source_path = r'F:\collection_spb'

In [3]:
xlsx_list = []
# 遍历文件夹下的xlsx文件
for root,dir,files in os.walk(source_path):
    for file in files:
        if file.endswith('.xlsx'):
            xlsx_list.append(os.path.join(root,file))

In [5]:
len(xlsx_list)

19

In [12]:
import pandas as pd


# 创建一个空的数据框，用于合并所有数据
result = pd.DataFrame()

# 遍历xlsx列表中的所有文件
for file in xlsx_list:
    # 读取当前文件的内容
    df = pd.read_excel(file)
    
    # 将当前文件的内容添加到结果数据框中
    result = pd.concat([result, df], ignore_index=True)



In [17]:
result.to_csv(r'F:\collection_spb\result.csv', index=False)

In [1]:
import arcpy
import os

# 设置工作空间和目标数据库
input_folder = r"C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml"
output_gdb = r"C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb"

# 创建目标数据库（如果不存在）
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

# 定义临时工作空间
scratch_folder = arcpy.env.scratchFolder

# 用于存储第一个数据库的坐标系
first_srid = None

# 遍历输入文件夹中的每个KML文件
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".kml") or file.endswith(".kmz"):
            kml_file = os.path.join(root, file)
            # 创建唯一临时GDB路径
            temp_gdb = arcpy.CreateUniqueName("temp.gdb", scratch_folder)
            arcpy.CreateFileGDB_management(scratch_folder, os.path.basename(temp_gdb))

            # 转换KML文件的输出文件夹路径
            output_layer_name = os.path.basename(kml_file).replace('.kml', '').replace('.kmz', '')
            output_layer_folder = os.path.join(scratch_folder, output_layer_name + ".gdb")

            # 删除可能存在的同名输出文件夹
            if arcpy.Exists(output_layer_folder):
                arcpy.Delete_management(output_layer_folder)

            # 将KML转换为临时GDB中的要素类
            arcpy.KMLToLayer_conversion(kml_file, scratch_folder, output_layer_name)
            arcpy.env.workspace = output_layer_folder

            # 获取临时GDB中的数据集
            datasets = arcpy.ListDatasets()

            # 遍历每个数据集
            for dataset in datasets:
                arcpy.env.workspace = os.path.join(output_layer_folder, dataset)

                # 获取数据集中的要素类列表
                feature_classes = arcpy.ListFeatureClasses()

                # 按要素类型进行分类并合并到目标GDB
                for fc in feature_classes:
                    shape_type = arcpy.Describe(fc).shapeType

                    # 获取第一个数据库的坐标系
                    if first_srid is None:
                        first_srid = arcpy.Describe(fc).spatialReference

                    # 如果当前要素类的坐标系不同于第一个数据库的坐标系，进行投影
                    if arcpy.Describe(fc).spatialReference.factoryCode != first_srid.factoryCode:
                        projected_fc = os.path.join(output_layer_folder, f"{fc}_projected")
                        arcpy.Project_management(fc, projected_fc, first_srid)
                        fc = projected_fc

                    output_fc_name = f"{shape_type}_{dataset}_{os.path.basename(fc)}"
                    output_fc = os.path.join(output_gdb, output_fc_name)
                    if not arcpy.Exists(output_fc):
                        # 创建新的要素类
                        arcpy.CreateFeatureclass_management(output_gdb, output_fc_name, shape_type, spatial_reference=first_srid)

                    # 合并要素类
                    arcpy.Append_management(fc, output_fc, "NO_TEST")
                    print(f"Features from {kml_file} dataset {dataset} have been merged into {output_fc}")

            # 删除临时GDB
            arcpy.Delete_management(temp_gdb)
print("KML文件转换并合并完成！")

Features from C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml\从江县丙妹镇占补平衡土地综合整治项目（一期）.kml dataset Placemarks have been merged into C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb\Point_Placemarks_Points
Features from C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml\从江县丙妹镇占补平衡土地综合整治项目（一期）.kml dataset Placemarks have been merged into C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb\Polygon_Placemarks_Polygons
Features from C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml\从江县丙梅街道占补平衡土地综合整治项目（一期）.kml dataset Placemarks have been merged into C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb\Point_Placemarks_Points
Features from C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml\从江县丙梅街道占补平衡土地综合整治项目（一期）.kml dataset Placemarks have been merged into C:\Users\Runker\Desktop\从江35个占补kml(1)\result.gdb\Polygon_Placemarks_Polygons
Features from C:\Users\Runker\Desktop\从江35个占补kml(1)\从江36个占补kml\从江县丙梅街道占补平衡土地综合整治项目（三期）.kml dataset Placemarks have been merged into C:\Users\Runker\Desktop\从江35个占补kml(1)\resu